### First step

In the following step we are going to find the best joint (bid, pricing) using the implementation of Non Linear Programming optimization Algorithm provided by the Scipy Library

In [1]:
from customerManager import CustomerManager
customer_path = "./config/customer_classes.json"
cManager = CustomerManager(file_path = customer_path)

In [2]:
import numpy as np
import scipy.optimize as opt

count = 0
def objective(x):   
    global count    
    count = count + 1

    #Checks bound of bid 
    res = 0.0
    price = x[3]
    for i in range(3):
        if x[i] < 1 or x[i] > 10 : return 0.0 
    
    for c in cManager.classes : 
        c_id = c["class_id"]
        bid = x[c_id]
        res += (price * cManager.sold_items(class_id = c_id, bid = bid, price = price, noise=False)) - \
              (cManager.cost_per_click(class_id=c_id, bid = bid, noise=False) * cManager.clicks(class_id = c_id, bid = bid,noise=False))
    return -res



# optimize
b_p = (3.0,15.0)
b_b = (1.0 , 10.0)
bnds = (b_b, b_b, b_b, b_p)

methods_with_jac = ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-krylov','trust-exact']
methods = ['Nelder-Mead','Powell','CG','BFGS','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr']

max_value = 0
best_method = 'none'
best_x = None

In [3]:
import warnings
warnings.filterwarnings("ignore")

# perfrom over different model
# initial guesses
n = 4
x0 = np.zeros(n)
x0[0] = 5.0    #Bid_class0
x0[1] = 5.0    #Bid_class1
x0[2] = 5.0    #Bid_class2
x0[3] = 7.5    #Price
# show initial objective
'''
for x0 in X0:
    print('Initial SSE Objective: ' + str(objective(x0)))
    print(x0)
'''
for m in methods:

    count = 0
    solution = opt.minimize(objective, x0, method = m,  bounds = bnds) 
    x = solution.x
    '''
    print("Start with " + m)
    print('Final SSE Objective: ' + str(-objective(x)))
    print('Objective function runned : ' + str(count))
    # print solution
    print('Solution')
    print('x1 = ' + str(x[0]))
    print('x2 = ' + str(x[1]))
    print('x3 = ' + str(x[2]))
    print('x4 = ' + str(x[3]))
    '''
    if max_value < -objective(x):
        max_value = -objective(x)
        best_method = m
        best_x = x

print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('x1 = ' + str(best_x[0]))
print('x2 = ' + str(best_x[1]))
print('x3 = ' + str(best_x[2]))
print('x4 = ' + str(best_x[3]))


Best method is Powell with a value of 5428.765711736162
Solution
x1 = 5.928979108120101
x2 = 2.6824056524184523
x3 = 3.1209461045533353
x4 = 6.534959661928547


In [4]:
#perform over different stating points
X0 = []
for i in range(1,8):
    n = float(i)
    if i >= 10:
        n = 9.0
    X0.append([n,n,n,float(i+3)])

for x0 in X0:
    count = 0
    solution = opt.minimize(objective, x0, method = 'Powell',  bounds = bnds) 
    x = solution.x
    if max_value < -objective(x):
        max_value = -objective(x)
        best_method = 'Powell'
        best_x = x

print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('x1 = ' + str(best_x[0]))
print('x2 = ' + str(best_x[1]))
print('x3 = ' + str(best_x[2]))
print('x4 = ' + str(best_x[3]))

Best method is Powell with a value of 5430.25897536469
Solution
x1 = 5.928954108179956
x2 = 2.682396521822428
x3 = 2.9540452274843294
x4 = 6.541655879916771
